In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!git clone https://github.com/anvaypandit/Emotion-Classification-Facial-Images-FER2013.git

Cloning into 'Emotion-Classification-Facial-Images-FER2013'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 41 (delta 14), reused 31 (delta 7), pack-reused 0
Unpacking objects: 100% (41/41), done.


In [3]:
cd Emotion-Classification-Facial-Images-FER2013/

/content/Emotion-Classification-Facial-Images-FER2013


In [0]:
mkdir models

In [5]:
!pip install Keras==2.2.0

    100% |████████████████████████████████| 307kB 9.2MB/s 
    100% |████████████████████████████████| 51kB 26.7MB/s 
tensorflow 1.13.1 has requirement keras-applications>=1.0.6, but you'll have keras-applications 1.0.2 which is incompatible.
tensorflow 1.13.1 has requirement keras-preprocessing>=1.0.5, but you'll have keras-preprocessing 1.0.1 which is incompatible.
  Found existing installation: Keras-Preprocessing 1.0.9
    Uninstalling Keras-Preprocessing-1.0.9:
      Successfully uninstalled Keras-Preprocessing-1.0.9
  Found existing installation: Keras-Applications 1.0.7
    Uninstalling Keras-Applications-1.0.7:
      Successfully uninstalled Keras-Applications-1.0.7
  Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


In [6]:
!pip install keras_vggface

    100% |████████████████████████████████| 61kB 3.0MB/s 
    100% |████████████████████████████████| 61kB 34.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/72/aa/01/eb7baeb2f6e2d2f0d2aabddb5f01d57fa22fbd019ee2799bf5
Successfully built keras-vggface
keras 2.2.0 has requirement keras-applications==1.0.2, but you'll have keras-applications 1.0.7 which is incompatible.
keras 2.2.0 has requirement keras-preprocessing==1.0.1, but you'll have keras-preprocessing 1.0.9 which is incompatible.
  Found existing installation: Keras-Preprocessing 1.0.1
    Uninstalling Keras-Preprocessing-1.0.1:
      Successfully uninstalled Keras-Preprocessing-1.0.1
  Found existing installation: Keras-Applications 1.0.2
    Uninstalling Keras-Applications-1.0.2:
      Successfully uninstalled Keras-Applications-1.0.2


In [10]:
pwd

'/content/Emotion-Classification-Facial-Images-FER2013'

In [11]:
!python image_preprocessor.py

Using TensorFlow backend.


In [12]:
import numpy as np
import pandas as pd
from tensorflow.python.lib.io import file_io
from skimage.transform import resize
from keras import backend as K
from keras.utils import to_categorical
from keras_vggface.vggface import VGGFace
from keras.models import Model
from keras.layers import Flatten, Dense 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam,SGD
from keras.callbacks import TensorBoard, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping, Callback,ModelCheckpoint
import h5py # For saving the model

Using TensorFlow backend.


In [0]:
# PARAMETERS ##########################################################################################################################################

# Folder where logs and models are stored
folder = 'ResNet-50'

# Size of the images
img_height, img_width = 197,197

# Parameters
num_classes         = 7     # ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise', 'Neutral']
epochs_top_layers   = 5
epochs_all_layers   = 100
batch_size          = 128

In [14]:
# MODEL ###############################################################################################################################################

# Create the based on ResNet-50 architecture pre-trained model
    # model:        Selects one of the available architectures vgg16, resnet50 or senet50
    # include_top:  Whether to include the fully-connected layer at the top of the network
    # weights:      Pre-training on VGGFace
    # input_shape:  Optional shape tuple, only to be specified if include_top is False (otherwise the input
    #               shape has to be (224, 224, 3) (with 'channels_last' data format) or (3, 224, 224) (with
    #               'channels_first' data format). It should have exactly 3 inputs channels, and width and
    #               height should be no smaller than 197. E.g. (200, 200, 3) would be one valid value.
# Returns a keras Model instance
base_model = VGGFace(
    model       = 'resnet50',
    include_top = False,
    weights     = 'vggface',
    input_shape = (img_height,img_width,3))

# Places x as the output of the pre-trained model
x = base_model.output

# Flattens the input. Does not affect the batch size
x = Flatten()(x)

# Add a fully-connected layer and a logistic layer
# Dense implements the operation: output = activation(dot(input, kernel) + bias(only applicable if use_bias is True))
    # units:        Positive integer, dimensionality of the output space
    # activation:   Activation function to use
    # input shape:  nD tensor with shape: (batch_size, ..., input_dim)
    # output shape: nD tensor with shape: (batch_size, ..., units)

#x = Dense(1024, activation = 'relu')(x)
x = Dense(1024, activation = 'relu')(x)
predictions = Dense(num_classes, activation = 'softmax')(x)

# The model we will train
model = Model(inputs = base_model.input, outputs = predictions)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
94699520/94694792 [==============================] - 1s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 197, 197, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 99, 99, 64)   9408        input_1[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 99, 99, 64)   256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 99, 99, 64)   0           conv1/

In [15]:
# Load Model 1
layer_name = 'dense_1'
model.load_weights('models/Model.40-0.6882.hdf5')
model2= Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

newtrain_datagen = ImageDataGenerator(
    samplewise_center=True,  # set each sample mean to 0
    rescale=(1/255.),
    data_format='channels_last')  # randomly flip images


newtrain_generator = newtrain_datagen.flow_from_directory(
        'data/Train',
        target_size=(197,197),
        batch_size = 1,
        class_mode=None,
        shuffle=False)

newtrain_generator.reset()

pred=model2.predict_generator(newtrain_generator,verbose=1,steps=28709)

Found 28709 images belonging to 7 classes.
28709/28709 [==============================] - 1403s 49ms/step


In [16]:
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 197, 197, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 99, 99, 64)   9408        input_1[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 99, 99, 64)   256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 99, 99, 64)   0           conv1/7x7_s2/bn[0][0]            
__________________________________________________________________________________________________
max_poolin

In [0]:
# Pickle the predictions to ensemble later
import pickle
pickle.dump(pred,open("predResnet50.pickle","wb"))

<class 'numpy.int32'>


In [0]:
import xgboost as xgb
import pickle
x_train = pred
y_train = newtrain_generator.labels
print('Training XGB Classifier from new features:')
xgb_model = xgb.XGBClassifier(max_depth=40,num_class=7, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='multi:softmax', eta=0.3, silent=0, subsample=0.8).fit(x_train, y_train)
#xgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8).fit(X_train, y_train)
#xgb_model = xgb.XGBClassifier().fit(x_train,y_train)
# Saving XGB model
print('Saving Model:')
#xgb_model.save_model('CharNgramXgBoost.model')
pickle.dump(xgb_model,open("XgBoostAfterResnet40D.pickle","wb"))

Training XGB Classifier from new features:


In [21]:
newtest_datagen = ImageDataGenerator(
    samplewise_center=True,  # set each sample mean to 0
    rescale=(1/255.),
    data_format='channels_last')  # randomly flip images


newtest_generator = newtrain_datagen.flow_from_directory(
        'data/Test',
        target_size=(197,197),
        batch_size = 1,
        class_mode=None,
        shuffle=False)

newtest_generator.reset()

x_test=model2.predict_generator(newtest_generator,verbose=1,steps=3589)
xgb_prediction = xgb_model.predict(x_test)

filenames_test=newtest_generator.filenames
results=pd.DataFrame({"Filename":filenames_test,
                      "Predictions":xgb_prediction})
results.to_csv("ResultsXGBTuned20D.csv",index=False)




Found 3589 images belonging to 1 classes.
3589/3589 [==============================] - 45s 12ms/step
